# 15_DTOs_Request_Body_Validation_and_Class_Validator

This part of your Nest.js note series explains:

* Why built-in pipes are **not enough** for validating complex request bodies
* What a **DTO (Data Transfer Object)** is
* How to create a **DTO class** for users
* How to add **validation rules** using the `class-validator` library

Everything here is based exactly on the lecture content you provided.

---

## 1. Why Built-in Pipes Are Not Enough for Request Body Validation

Earlier, we learned how to use **built-in pipes** like:

* `ParseIntPipe`
* `DefaultValuePipe`

to validate and transform **query strings** and **route parameters**.

Example recap:

* Validating **query strings** using `DefaultValuePipe` and `ParseIntPipe`
* Validating **route parameters** using `ParseIntPipe`

### 1.1 Built-in pipes work well for:

* **Simple data**
* Small amount of data passed via:

  * **Query strings** (e.g. `?limit=10&page=2`)
  * **Route parameters** (e.g. `/users/1`)

👉 For these, built-in pipes are **perfect**:

* They can easily convert and validate primitive values (string → number, etc.).

### 1.2 The problem: Validating Request Bodies

But request bodies (especially in **POST** or **PUT** requests) often contain:

* **Large, complex objects**
* Many properties
* Different data types

Example: Creating a user via POST:

* URL: `POST /users`
* Request body (`user` object):

  ```json
  {
    "id": 1,
    "name": "John",
    "email": "john@example.com",
    "gender": "male",
    "isMarried": false
  }
  ```

In real-world applications, the `user` object might also include:

* `phoneNumber`
* `city`
* `password`
* etc.

As the object grows:

* Validating each field **individually with built-in pipes** becomes:

  * Hard to manage
  * Very cluttered
  * Difficult to maintain

### 1.3 Need for better validation of request body

We need a better way to:

* Validate the entire **request body** before creating a resource in the database.
* Prevent invalid data from being saved.

Example problems:

* If `name` is missing or an empty string:

  * Currently the server still creates the user → **this is wrong**.
* We should treat that as **invalid data** and **reject the request**.

Other examples of required validations:

* **Email** must be a valid email format
* **Phone number** must be valid
* **Password** must:

  * Have a minimum length (e.g. 8 characters)
  * Optionally contain numbers, special characters, etc.

✅ Writing all of this validation logic manually with built-in pipes is **complex and ugly**.

👉 Solution: Use **DTOs + class-validator** + (later) `ValidationPipe`.

---

## 2. DTOs – Data Transfer Objects

### 2.1 What is a DTO?

**DTO = Data Transfer Object**

* A **simple TypeScript class** used to represent the data:

  * being transferred between different layers of the app:

    * **Controller**
    * **Service**
    * **Repository** (or data access layer)
* Helps to:

  * **Decouple layers**
  * Keep code more **readable** and **maintainable**

> For now, the key idea: **A DTO is just a class that defines the shape of the data and the rules for it.**

### 2.2 How we use a DTO

* We define a DTO class with all the properties that we expect in the request body.
* We will later connect this DTO to the controller method parameter.
* We attach **validation decorators** to its properties to enforce rules.
* Before the controller logic runs, Nest can validate the incoming request body **against this DTO**.

---

## 3. Creating a DTO for Creating a User

We’ll create a DTO dedicated to **creating a user**.

> We can have different DTOs for different operations:
>
> * `CreateUserDto` – for creating a user
> * `UpdateUserDto` – for updating a user
> * `FindUserDto` – for filtering/searching users

### 3.1 Folder structure for DTOs

Inside the `users` module/folder, we create a directory to store all DTOs related to users:

* `users/`

  * `dtos/`

    * `create-user.dto.ts`

You can name the folder anything, but using **`dtos`** makes it very clear.

### 3.2 Defining the `CreateUserDto` class

File: `users/dtos/create-user.dto.ts`

```ts
export class CreateUserDto {
  id: number;
  name: string;
  gender: string;
  email: string;
  isMarried: boolean;
}
```

Here:

* DTO is just a **class** with properties.
* These properties represent what we expect in the **request body** when creating a user.

### 3.3 Matching DTO with existing user structure

In `user.service.ts` we might have a user list like:

```ts
users = [
  {
    id: 1,
    name: 'John',
    email: 'john@example.com',
    gender: 'male',
    isMarried: false,
  },
  // ...
];
```

> We updated the user model to use `email` instead of `age`, and adjusted the type to `string`.

The DTO mirrors those properties:

* `id: number`
* `name: string`
* `gender: string`
* `email: string`
* `isMarried: boolean`

Now we have:

* A clear **shape** for incoming request data.
* Next step: add **validation rules** to these properties.

---

## 4. Why We Need Validation on DTO Properties

Before we connect the DTO to the request body, we want to:

* Validate that:

  * `name` has a proper string value and is **not empty**
  * `id` is a **number**
  * `email` is a **valid email address**
  * `gender` is optional but, if provided, must be a **string**

We don’t want to:

* Accept empty strings for `name`.
* Accept invalid email addresses.
* Accept wrong data types (e.g. numbers for `name`).

To perform these validations, we use a third-party library: **`class-validator`**.

---

## 5. Installing `class-validator`

### 5.1 What is `class-validator`?

* A third-party library that allows:

  * **Decorator-based** validation
  * **Non-decorator-based** validation (we focus on decorators)
* It internally uses **`validator.js`** under the hood to perform many validations.
* Works in both:

  * **Browser**
  * **Node.js**

We’ll use `class-validator` to attach **validation decorators** to DTO properties.

### 5.2 Installing the package

1. Open **npmjs.com**, search for `class-validator`.

2. Note the version used in the lecture: **`0.14.1`**.

   * If you’re following the course exactly, install **this version**.
   * In the future, new versions may behave slightly differently.

3. In VS Code terminal:

```bash
npm install class-validator@0.14.1
```

* If you omit the version, npm will install the latest version.
* To ensure compatibility with the lecture, use `@0.14.1`.

4. After install, check `package.json`:

   * Under `dependencies`, you should see:

     * `"class-validator": "0.14.1"`

✅ Now `class-validator` is ready to use.

---

## 6. Adding Validation Decorators to the DTO

We now add validation rules to `CreateUserDto` using decorators from `class-validator`.

### 6.1 Basic string validations

We want `name`, `gender`, and `email` to be **string values**.

We can use the `@IsString()` decorator.

```ts
import { IsString, IsEmail, IsOptional, IsNotEmpty, MinLength } from 'class-validator';

export class CreateUserDto {
  id: number;

  @IsString({ message: 'Name should be a string value' })
  @IsNotEmpty({ message: 'Name should not be empty' })
  @MinLength(3, { message: 'Name should have a minimum of three characters' })
  name: string;

  @IsOptional()
  @IsString({ message: 'Gender should be a string value' })
  gender?: string;

  @IsString({ message: 'Email should be a string value' })
  @IsEmail({}, { message: 'Email must be a valid email address' })
  email: string;

  isMarried: boolean;
}
```

> The above combines all the validations demonstrated in the lecture.

### 6.2 Decorators used (and their purpose)

1. **`@IsString()`**

   * Ensures the property value is a **string**.
   * Used on: `name`, `gender`, `email`.

2. **`@IsEmail()`**

   * Ensures the property is a **valid email address**.
   * Used on: `email`.
   * Also implies it must be a string.

3. **`@IsOptional()`**

   * Marks the property as **optional**.
   * If the value is `undefined` or not provided:

     * The other validators on that property are **not executed**.
   * Used on: `gender`.
   * In TypeScript, we also add `?`:

     ```ts
     gender?: string;
     ```

4. **`@IsNotEmpty()`**

   * Ensures the string is **not empty** (`''`).
   * Used on: `name`.
   * This prevents cases like `"name": ""` from being considered valid.

5. **`@MinLength()`**

   * Ensures the string has at least a certain number of characters.
   * Used on: `name`.
   * Example:

     ```ts
     @MinLength(3)
     ```

     → name must be at least **3 characters**.

### 6.3 Discovering other validators

`class-validator` provides **many** other decorators, such as:

* `@IsAlpha()` / `@IsAlphanumeric()`
* `@IsArray()`
* `@IsCreditCard()`
* `@IsCurrency()`
* `@IsDate()`
* `@MaxLength()` / `@MinLength()`
* `@Max()` / `@Min()`

You can explore them by typing `@Is` and letting IntelliSense suggest options.

> In the lecture, several examples were mentioned: `IsString`, `IsNotEmpty`, `IsOptional`, `IsEmail`, `MinLength`, `MaxLength`, `Min`, `Max`, etc.

For now, we keep it **simple**, using only what we need for the `CreateUserDto`.

---

## 7. Custom Error Messages

Each validator decorator can accept an **options object** where you can define a **custom error message**.

Example:

```ts
@IsString({ message: 'Name should be a string value' })
name: string;

@MinLength(3, { message: 'Name should have a minimum of three characters' })
name: string;
```

* If validation fails, these custom messages will appear in the validation error response.
* If you **don’t** specify a message:

  * A default, built-in message from `class-validator` will be used.

You can do this for **all validators** (e.g., `IsEmail`, `IsNotEmpty`, etc.).

---

## 8. Summary of This Part

In this part of your Nest.js notes, we covered:

1. **Limitations of built-in pipes** for validating complex request bodies.
2. The need for a better structure for body validation → **DTOs**.
3. **DTO (Data Transfer Object)**:

   * A simple TypeScript class that defines the shape of the data.
   * Used between controllers, services, and repositories.
4. Created a **`CreateUserDto`** class with properties:

   * `id`, `name`, `gender`, `email`, `isMarried`.
5. Installed and used **`class-validator`** (`0.14.1`):

   * Added validation decorators to DTO properties:

     * `@IsString()`
     * `@IsNotEmpty()`
     * `@IsOptional()`
     * `@IsEmail()`
     * `@MinLength()`
6. Learned that each decorator can:

   * Enforce a specific rule
   * Optionally have a **custom error message**.
7. We kept validation **simple** for now, but `class-validator` supports many more decorators for complex rules.

---

This will complete the flow:

* Client → Request body → DTO + class-validator → ValidationPipe → Controller → Service → Database.
